In [109]:
import pickle
import glob
import networkx as nx 
import matplotlib.pyplot as plt
import math

to_classify = ["sample", "domain", "phylum", "class", "order", "family", "genus"]

disease = "crc"
classifier = 'family'
filepath = "results/file_dicts/"+disease+"_"+classifier+".pkl"

In [110]:
with open(filepath, 'rb') as f:
    total_files_per_sample = pickle.load(f)

num_files = len(glob.glob('results/'+disease+'/'+disease+'_dicts/*'))

In [111]:
# graphing

sum = 0
total = 0

for key in total_files_per_sample:
    for key2 in total_files_per_sample[key]:
        sum += total_files_per_sample[key][key2]
        total +=1

if total == 0:
    print("Error: no correlation")
avg_tfs = sum / total
print(avg_tfs, num_files)

min = 1

graph = nx.Graph()
used = []
for node in total_files_per_sample:
    if node not in used:
        graph.add_node(node, node_size=0)
    used.append(node)    

for node in total_files_per_sample:
    for other in total_files_per_sample[node]:
        if node != other and total_files_per_sample[node][other] > avg_tfs * 2:    
            # print(total_files_per_sample[node][other] / num_files)         
            graph.add_edge(node, other, weight = (total_files_per_sample[node][other] / num_files), color = 'black')
            if ((total_files_per_sample[node][other] / num_files) < min):
                min = (total_files_per_sample[node][other] / num_files)

min += 0.001

7.237075190756764 42


In [114]:
plt.figure(figsize=(15,15), dpi = 600)

# spring = nx.spring_layout(graph, seed=7)
circ = nx.circular_layout(graph)

nx.draw_networkx_nodes(graph, circ, node_size = 1)
for (u, v, d) in graph.edges(data = True):
    nx.draw_networkx_edges(graph, circ, edgelist=[(u, v)], width = ((d["weight"] - min)/ 0.1)**2)

# plt.title("Graph of connections between " + classifier + " for disease " + disease.upper(), fontsize = 25)

for pos in ['right', 'top', 'bottom', 'left']: 
    plt.gca().spines[pos].set_visible(False) 

plt.show()

### Weighted graph analysis
calculate the characteristic path length of a network: the average shortest graph between any two nodes

In [106]:
def calc_weight(u, v, d):
    # print( ((d["weight"])/ 0.1)**2)
    return math.exp(d["weight"] * -1)

def calc_weight_inv(d):
    return d["weight"] / num_files

In [112]:
# remove vertices with no neighbors
graph.remove_nodes_from(list(nx.isolates(graph)))
avg_path_len_uw = nx.average_shortest_path_length(graph) # / len(graph.nodes)
avg_path_len_w = nx.average_shortest_path_length(graph, weight=calc_weight)

print("average unweighted path length:", avg_path_len_uw)
print("average weighted path length:", avg_path_len_w)
print("ratio:", avg_path_len_w / avg_path_len_uw)

average unweighted path length: 1.749078397640698
average weighted path length: 1.1522708570013562
ratio: 0.6587874268847153


strength of weighted degree

In [113]:
total_sum = 0
for node in graph.nodes:
    sum = 0
    edges = graph.edges(node, data = True)
    for edge in edges:
        sum += calc_weight_inv(edge[2])
    total_sum += sum

print("average:", total_sum / len(graph.nodes))

average: 0.8978613809741145
